# Tarefa 4

<b>Alunos:</b> Andreza (164213), Gil (225323) e Yan (118982)

In [3]:
import numpy as np
import random
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, cross_val_score
from hyperopt import tpe, fmin, hp, STATUS_OK
from pyswarm import pso
from simanneal import Annealer
import optuna

## Hiperparâmetros:
* C: 2^{-5} a 2^{15} (uniforme nos expoentes)
* gamma: 2^{-15} a 2^{3} (uniforme nos expoentes)
* epsilon: 0.05 a 1.0 (uniforme no intervalo)

## Carregamento dos dados

In [4]:
X = np.load("X.npy")
y = np.load("y.npy")

## Random Search

In [5]:
C_values = [2**float(expo) for expo in np.random.uniform(low = -5, high = 15, size = 125)]
gamma_values = [2**float(expo) for expo in np.random.uniform(low = -15, high = 3, size = 125)]
param_distributions = {'C': C_values, 
                       'gamma': gamma_values ,
                       'epsilon': np.random.uniform(low = 0.05, high = 1.0, size = 125)}

# Realiza a busca aleatória
# Se cv = None, utiliza 5 fold cross validation
random_search = RandomizedSearchCV(
    SVR(),
    param_distributions,
    random_state = 0,
    n_iter=125,
    scoring = 'neg_mean_squared_error',
    cv = None,
    refit = True,
    verbose=2)
random_search.fit(X, y)

Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365 
[CV]  gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365, total=   0.0s
[CV] gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365 
[CV]  gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365, total=   0.0s
[CV] gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365 
[CV]  gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365, total=   0.0s
[CV] gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365 
[CV]  gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365, total=   0.0s
[CV] gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365 
[CV]  gamma=3.0246233162893548, epsilon=0.7271248424072555, C=180.5135606404365, total=   0.0s
[CV] gamma=6.580102801581101, epsilon=0.9795315384867214, C=17

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  gamma=0.02780596680000636, epsilon=0.3375509820899817, C=0.372775046834544, total=   0.0s
[CV] gamma=0.02780596680000636, epsilon=0.3375509820899817, C=0.372775046834544 
[CV]  gamma=0.02780596680000636, epsilon=0.3375509820899817, C=0.372775046834544, total=   0.0s
[CV] gamma=0.02780596680000636, epsilon=0.3375509820899817, C=0.372775046834544 
[CV]  gamma=0.02780596680000636, epsilon=0.3375509820899817, C=0.372775046834544, total=   0.0s
[CV] gamma=3.57752239193278e-05, epsilon=0.3803324704699518, C=68.40117729899963 
[CV]  gamma=3.57752239193278e-05, epsilon=0.3803324704699518, C=68.40117729899963, total=   0.0s
[CV] gamma=3.57752239193278e-05, epsilon=0.3803324704699518, C=68.40117729899963 
[CV]  gamma=3.57752239193278e-05, epsilon=0.3803324704699518, C=68.40117729899963, total=   0.0s
[CV] gamma=3.57752239193278e-05, epsilon=0.3803324704699518, C=68.40117729899963 
[CV]  gamma=3.57752239193278e-05, epsilon=0.3803324704699518, C=68.40117729899963, total=   0.0s
[CV] gamma=3.

[CV]  gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182, total=   1.3s
[CV] gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182 
[CV]  gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182, total=   1.2s
[CV] gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182 
[CV]  gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182, total=   1.6s
[CV] gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182 
[CV]  gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182, total=   1.2s
[CV] gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182 
[CV]  gamma=0.0001891928161533287, epsilon=0.09452550810244872, C=24809.905165565182, total=   1.4s
[CV] gamma=2.4810695434881644, epsilon=0.9696689758396094, C=0.04962207810889675 
[CV]  gamma=2.4810695434881644, epsilon=0.9696689758396094, C=0.04962207810889

[CV]  gamma=0.499549286349868, epsilon=0.7312988103669412, C=469.9519664941214, total=   0.0s
[CV] gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367 
[CV]  gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367, total=   0.0s
[CV] gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367 
[CV]  gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367, total=   0.0s
[CV] gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367 
[CV]  gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367, total=   0.0s
[CV] gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367 
[CV]  gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367, total=   0.0s
[CV] gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078367 
[CV]  gamma=0.00015333137735257243, epsilon=0.7912457279332973, C=243.01213385078

...

[CV]  gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344, total=   0.0s
[CV] gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344 
[CV]  gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344, total=   0.0s
[CV] gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344 
[CV]  gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344, total=   0.0s
[CV] gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344 
[CV]  gamma=0.0007930875999181205, epsilon=0.7271248424072555, C=230.29568254020344, total=   0.0s
[CV] gamma=0.0007930875999181205, epsilon=0.3375509820899817, C=2.348973005515618 
[CV]  gamma=0.0007930875999181205, epsilon=0.3375509820899817, C=2.348973005515618, total=   0.0s
[CV] gamma=0.0007930875999181205, epsilon=0.3375509820899817, C=2.348973005515618 
[CV]  gamma=0.0007930875999181205, epsilon=0.3375509820899817, C=2.348973005515618, tota

[CV]  gamma=0.000530283182480436, epsilon=0.7924522443332849, C=2723.837003026699, total=   0.2s
[CV] gamma=0.000530283182480436, epsilon=0.7924522443332849, C=2723.837003026699 
[CV]  gamma=0.000530283182480436, epsilon=0.7924522443332849, C=2723.837003026699, total=   0.2s
[CV] gamma=0.000530283182480436, epsilon=0.7924522443332849, C=2723.837003026699 
[CV]  gamma=0.000530283182480436, epsilon=0.7924522443332849, C=2723.837003026699, total=   0.2s
[CV] gamma=0.0009270322629123327, epsilon=0.8831265046081439, C=0.09242423070442499 
[CV]  gamma=0.0009270322629123327, epsilon=0.8831265046081439, C=0.09242423070442499, total=   0.0s
[CV] gamma=0.0009270322629123327, epsilon=0.8831265046081439, C=0.09242423070442499 
[CV]  gamma=0.0009270322629123327, epsilon=0.8831265046081439, C=0.09242423070442499, total=   0.0s
[CV] gamma=0.0009270322629123327, epsilon=0.8831265046081439, C=0.09242423070442499 
[CV]  gamma=0.0009270322629123327, epsilon=0.8831265046081439, C=0.09242423070442499, tota

[CV]  gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339, total=   0.0s
[CV] gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339 
[CV]  gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339, total=   0.0s
[CV] gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339 
[CV]  gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339, total=   0.0s
[CV] gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339 
[CV]  gamma=0.4747310069854422, epsilon=0.5279562184452645, C=6112.204928850339, total=   0.0s
[CV] gamma=7.1208002441740454, epsilon=0.27163405414900227, C=2604.7139694781736 
[CV]  gamma=7.1208002441740454, epsilon=0.27163405414900227, C=2604.7139694781736, total=   0.0s
[CV] gamma=7.1208002441740454, epsilon=0.27163405414900227, C=2604.7139694781736 
[CV]  gamma=7.1208002441740454, epsilon=0.27163405414900227, C=2604.7139694781736, total=   0.0s
[CV] gamma=7.120800244

[Parallel(n_jobs=1)]: Done 625 out of 625 | elapsed:   18.2s finished


RandomizedSearchCV(estimator=SVR(), n_iter=125,
                   param_distributions={'C': [0.09242423070442499,
                                              32.14576415936607,
                                              1.94980228794557,
                                              8513.949420358162,
                                              673.5398584211908,
                                              11.465432691944887,
                                              6112.204928850339,
                                              0.2600580019161118,
                                              4875.336244148578,
                                              0.372775046834544,
                                              29714.715871807977,
                                              18437.78050156187,
                                              0.08164495726632773,
                                              234.15979007013104,
                                   

In [6]:
print("Melhores parâmetros: ", random_search.best_params_)
regressor = SVR(kernel = 'rbf', C = random_search.best_params_['C'],
                gamma = random_search.best_params_['gamma'],
                epsilon = random_search.best_params_['epsilon'])

scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Melhores parâmetros:  {'gamma': 5.167329881585846e-05, 'epsilon': 0.34009123901247107, 'C': 2723.837003026699}
RMSE: 4.149202656596389


## Grid Search

Fazemos um grid search com grid de 5x5x5 selecionando uniformemente 5 expoentes de C e gamma e 5 valores de epsilon

In [7]:
C_values = [2**float(expo) for expo in np.random.uniform(low = -5, high = 15, size = 5)]
gamma_values = [2**float(expo) for expo in np.random.uniform(low = -15, high = 3, size = 5)]
param_grid = {'C': C_values, 
              'gamma': gamma_values ,
              'epsilon': np.linspace(start = 0.05, stop = 1.0, num = 5)}
print(param_grid)

# Realiza o Grid Search utilizando os parametros definidos em param_grid
grid_search = GridSearchCV(SVR(),
                           param_grid,
                           cv = None,
                           scoring = 'neg_mean_squared_error',
                           refit = True,
                           verbose = 2)
grid_search.fit(X, y)

{'C': [4.824507773872936, 4148.675216442623, 1.249246235335057, 0.04056643581065036, 46.13294200070603], 'gamma': [0.005101164037607318, 3.2180102294831068, 0.22659193476588393, 1.4554775609227206, 0.010820979763324884], 'epsilon': array([0.05  , 0.2875, 0.525 , 0.7625, 1.    ])}
Fitting 5 folds for each of 125 candidates, totalling 625 fits
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.005101164037607318, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884 ...
[CV]  C=4.824507773872936, epsilon=0.05, gamma=0.010820979763324884, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.2875, gamma=0.005101164037607318 .
[CV]  C=4.824507773872936, epsilon=0.2875, gamma=0.005101164037607318, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.2875, gamma=0.005101164037607318 .
[CV]  C=4.824507773872936, epsilon=0.2875, gamma=0.005101164037607318, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.2875, gamma=0.005101164037607318 .
[CV]  C=4.824507773872936, epsilon=0.2875, gamma=0.005101164037607318, t

[CV]  C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=4.824507773872936, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.7625, gamma=0.010820979763324884 .
[CV]  C=4.824507773872936, epsilon=0.7625, gamma=0.010820979763324884, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.7625, gamma=0.010820979763324884 .
[CV]  C=4.824507773872936, epsilon=0.7625, gamma=0.010820979763324884, total=   0.0s
[CV] C=4.824507773872936, epsilon=0.7625, gamma=0.010820979763324884 .
[CV]  C=4.824507773872936, epsilon=0.7625, gamma=0.010820979763324884, t

[CV]  C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318, total=   0.0s
[CV] C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318 .
[CV]  C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318, total=   0.0s
[CV] C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318 .
[CV]  C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318, total=   0.0s
[CV] C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318 .
[CV]  C=4148.675216442623, epsilon=0.2875, gamma=0.005101164037607318, total=   0.0s
[CV] C=4148.675216442623, epsilon=0.2875, gamma=3.2180102294831068 ...
[CV]  C=4148.675216442623, epsilon=0.2875, gamma=3.2180102294831068, total=   0.0s
[CV] C=4148.675216442623, epsilon=0.2875, gamma=3.2180102294831068 ...
[CV]  C=4148.675216442623, epsilon=0.2875, gamma=3.2180102294831068, total=   0.0s
[CV] C=4148.675216442623, epsilon=0.2875, gamma=3.2180102294831068 ...
[CV]  C=4148.675216442623, epsilon=0.2875, gamma=3.2180102294831068,

...

[CV]  C=0.04056643581065036, epsilon=0.7625, gamma=0.010820979763324884, total=   0.0s
[CV] C=0.04056643581065036, epsilon=0.7625, gamma=0.010820979763324884 
[CV]  C=0.04056643581065036, epsilon=0.7625, gamma=0.010820979763324884, total=   0.0s
[CV] C=0.04056643581065036, epsilon=0.7625, gamma=0.010820979763324884 
[CV]  C=0.04056643581065036, epsilon=0.7625, gamma=0.010820979763324884, total=   0.0s
[CV] C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318 ..
[CV]  C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318, total=   0.0s
[CV] C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318 ..
[CV]  C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318, total=   0.0s
[CV] C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318 ..
[CV]  C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318, total=   0.0s
[CV] C=0.04056643581065036, epsilon=1.0, gamma=0.005101164037607318 ..
[CV]  C=0.04056643581065036, epsilon=1.0, gamma=0.005101164

[CV]  C=46.13294200070603, epsilon=0.2875, gamma=3.2180102294831068, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.2875, gamma=3.2180102294831068 ...
[CV]  C=46.13294200070603, epsilon=0.2875, gamma=3.2180102294831068, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393 ..
[CV]  C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393 ..
[CV]  C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393 ..
[CV]  C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393 ..
[CV]  C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393 ..
[CV]  C=46.13294200070603, epsilon=0.2875, gamma=0.22659193476588393, to

[CV]  C=46.13294200070603, epsilon=0.7625, gamma=0.22659193476588393, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206 ...
[CV]  C=46.13294200070603, epsilon=0.7625, gamma=1.4554775609227206, total=   0.0s
[CV] C=46.13294200070603, epsilon=0.7625, gamma=0.010820979763324884 .
[CV]  C=46.13294200070603, epsilon=0.7625, gamma=0.010820979763324884, tota

[Parallel(n_jobs=1)]: Done 625 out of 625 | elapsed:    6.9s finished


GridSearchCV(estimator=SVR(),
             param_grid={'C': [4.824507773872936, 4148.675216442623,
                               1.249246235335057, 0.04056643581065036,
                               46.13294200070603],
                         'epsilon': array([0.05  , 0.2875, 0.525 , 0.7625, 1.    ]),
                         'gamma': [0.005101164037607318, 3.2180102294831068,
                                   0.22659193476588393, 1.4554775609227206,
                                   0.010820979763324884]},
             scoring='neg_mean_squared_error', verbose=2)

In [8]:
print("Melhores parâmetros: ", grid_search.best_params_)
regressor = SVR(kernel = 'rbf', C = grid_search.best_params_['C'],
                gamma = grid_search.best_params_['gamma'],
                epsilon = grid_search.best_params_['epsilon'])

scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Melhores parâmetros:  {'C': 46.13294200070603, 'epsilon': 0.7625, 'gamma': 0.005101164037607318}
RMSE: 6.059970848451875


## Bayesian Search

A otimização bayesiana utilizando a bibliotec hyperopt requer a definição de uma função objetivo 
a ser minimizada. Como queremos maximizar a accuracia, utilizamos loss = -acc para ser minimizada. 

In [9]:
def objective_func(search_space):
    C =  search_space['C']
    gamma = search_space['gamma']
    epsilon = search_space['epsilon']
    regressor = SVR(**{'C': 2**C, 'gamma': 2**gamma, 'epsilon': epsilon})
    
    acc = cross_val_score(regressor, X, y, scoring='neg_mean_squared_error', cv=None).mean()
    return {'loss': -acc, 'status': STATUS_OK}

In [10]:
bayesian_search_space = {
    'C': hp.uniform('C', -5, 15),
    'gamma': hp.uniform('gamma', -15, 3),
    'epsilon': hp.uniform('epsilon', 0.05, 1.0)   
}

best_params = fmin(objective_func, bayesian_search_space, algo=tpe.suggest, max_evals=125)
best_parameters = {
    'C': 2**best_params['C'],
    'gamma': 2**best_params['gamma'],
    'epsilon': best_params['epsilon']
}

100%|██████████| 125/125 [01:12<00:00,  1.72trial/s, best loss: 15.682082756720328]


In [11]:
print("Melhores parâmetros:", best_parameters)
regressor = SVR(C=best_parameters['C'], gamma=best_parameters['gamma'], epsilon=best_parameters['epsilon'])
scores = cross_val_score(regressor, X, y, scoring = 'neg_mean_squared_error', cv=5)
rmse =  np.sqrt(np.mean(np.absolute(scores)))
print("RMSE:", rmse)

Melhores parâmetros: {'C': 7032.1595024598555, 'gamma': 3.8427918347574665e-05, 'epsilon': 0.3438598450002335}
RMSE: 3.960060953662245


## PSO

In [12]:
def funcao(param):  
    svr = SVR(kernel='rbf', C=param[0], gamma=param[1], epsilon=param[2])
    rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)
    scores = cross_val_score(svr, X, y, cv=5, scoring=rmse)#'neg_root_mean_squared_error')
    rmse_mean = -np.mean(scores)
    return rmse_mean

lb = np.array([2**(-5),2**(-15),0.05])
ub = np.array([2**15,2**3,1.0])

xopt, fopt = pso(funcao, lb, ub, swarmsize=11, maxiter=11)

C_opt = str(xopt[0])
gamma_opt = str(xopt[1])
epsilon_opt = str(xopt[2])

print("C ótimo: " + C_opt)
print("gamma ótimo: " + gamma_opt)
print("epsilon ótimo: " + epsilon_opt)
print("RMSE: " + str(fopt))

Stopping search: maximum iterations reached --> 11
C ótimo: 21682.636871809897
gamma ótimo: 3.0517578125e-05
epsilon ótimo: 0.05
RMSE: 3.7165230704898455


## Simulated annealing

In [19]:
class SimAnn(Annealer):

    def move(self):
        self.state[0] = random.uniform(2**exp_min_C, 2**exp_max_C)
        self.state[1] = random.uniform(2**exp_min_gamma, 2**exp_max_gamma)
        self.state[2] = random.uniform(min_epsilon, max_epsilon)
    
    def energy(self):
        svr = SVR(kernel='rbf', C=self.state[0], gamma=self.state[1], epsilon=self.state[2])
        rmse = make_scorer(mean_squared_error, squared=False, greater_is_better=False)
        scores = cross_val_score(svr, X, y, cv=5, scoring=rmse)#'neg_root_mean_squared_error')
        rmse_mean = -np.mean(scores)
        
        return rmse_mean

exp_min_C = -5
exp_max_C = 15
exp_min_gamma = -15
exp_max_gamma = 3
min_epsilon = 0.05
max_epsilon = 1.0
C = random.uniform(2**exp_min_C, 2**exp_max_C)
gamma = random.uniform(2**exp_min_gamma, 2**exp_max_gamma)
epsilon = random.uniform(min_epsilon, max_epsilon)
init_state = [C, gamma, epsilon]
sa = SimAnn(init_state)
sa.steps = 125
best_params, rmse = sa.anneal()
print('C ótimo: ' + str(best_params[0]))
print('gamma ótimo: ' + str(best_params[1]))
print('epsilon ótimo: ' + str(best_params[2]))
print('RMSE: ' + str(rmse))

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          9.14   100.00%   100.00%     0:00:05     0:00:00

C ótimo: 6906.718818535727
gamma ótimo: 0.06061829340519563
epsilon ótimo: 0.4677504822991566
RMSE: 8.037158980536786


## CMA-ES

Semelhantemente ao algoritmo de Simullate Annealing, a implementação do algoritmo de otimização CMA-ES também foi feita via biblioteca optunza. A implementação porém é encapsulada em um sampler default fornecido pela própria biblioteca, assim não sendo necessário a implementação de uma classe sampler específica, somente a função objetivo.

Utilize 125 chamadas da função.

In [14]:
def objective(trial):
    c = trial.suggest_loguniform('c', 2e-5,  2e15)
    gamma = trial.suggest_loguniform('gamma', 2e-15,  2e3)    
    epsilon = trial.suggest_uniform('epsilon', 0.05,  1.0)
    
    svr  = SVR(kernel='rbf', C=c, gamma=gamma, epsilon=epsilon)
    scores = cross_val_score(svr, X, y, scoring = 'neg_mean_squared_error', cv=5)
    rmse =  np.sqrt(np.mean(np.absolute(scores)))   
    return rmse


sampler = optuna.samplers.CmaEsSampler()
cmaes = optuna.create_study(sampler=sampler)
cmaes.optimize(objective, n_trials=125)

[I 2021-05-01 17:43:08,203] A new study created in memory with name: no-name-d10da896-9fba-41aa-9cca-e216edcc85cc
[I 2021-05-01 17:43:08,342] Trial 0 finished with value: 5.235755769931081 and parameters: {'c': 10533.184443726745, 'gamma': 1.0630135114349908e-06, 'epsilon': 0.15949029714396512}. Best is trial 0 with value: 5.235755769931081.
[I 2021-05-01 17:43:15,631] Trial 1 finished with value: 3.852333357108757 and parameters: {'c': 252881.86802121447, 'gamma': 2.2977006878170917e-06, 'epsilon': 0.45585494899591955}. Best is trial 1 with value: 3.852333357108757.
[I 2021-05-01 17:43:18,266] Trial 2 finished with value: 4.097111172078839 and parameters: {'c': 169770.92257929142, 'gamma': 1.8268175023293677e-06, 'epsilon': 0.8238275153140739}. Best is trial 1 with value: 3.852333357108757.
[I 2021-05-01 17:43:22,591] Trial 3 finished with value: 3.960796288330416 and parameters: {'c': 188836.65484921896, 'gamma': 2.066239693042443e-06, 'epsilon': 0.669599163819058}. Best is trial 1 w

[I 2021-05-01 17:46:34,433] Trial 34 finished with value: 3.6544430416794267 and parameters: {'c': 346896.0570887363, 'gamma': 3.540672806465678e-06, 'epsilon': 0.5858575163623129}. Best is trial 31 with value: 3.6001350939219723.
[I 2021-05-01 17:46:39,197] Trial 35 finished with value: 3.8696367810314745 and parameters: {'c': 188663.00423888367, 'gamma': 2.5379222576968574e-06, 'epsilon': 0.37333863936385947}. Best is trial 31 with value: 3.6001350939219723.
[I 2021-05-01 17:47:09,820] Trial 36 finished with value: 3.5876311632044025 and parameters: {'c': 464123.9921723557, 'gamma': 6.6370636511887045e-06, 'epsilon': 0.5661950764631961}. Best is trial 36 with value: 3.5876311632044025.
[I 2021-05-01 17:47:15,243] Trial 37 finished with value: 3.736267924479264 and parameters: {'c': 144085.6868818695, 'gamma': 4.637532116746745e-06, 'epsilon': 0.6745537157084804}. Best is trial 36 with value: 3.5876311632044025.
[I 2021-05-01 17:47:24,890] Trial 38 finished with value: 3.6819816068312

[I 2021-05-01 18:02:21,218] Trial 68 finished with value: 3.544958447150941 and parameters: {'c': 984560.5098462715, 'gamma': 3.149820830313796e-06, 'epsilon': 0.8660306384948733}. Best is trial 44 with value: 3.5193079556416222.
[I 2021-05-01 18:02:38,986] Trial 69 finished with value: 3.650191269236618 and parameters: {'c': 390685.130178278, 'gamma': 3.9122926085012555e-06, 'epsilon': 0.09463528859656634}. Best is trial 44 with value: 3.5193079556416222.
[I 2021-05-01 18:02:59,228] Trial 70 finished with value: 3.667190495061862 and parameters: {'c': 503202.92009143316, 'gamma': 2.834838794013176e-06, 'epsilon': 0.5094097946547103}. Best is trial 44 with value: 3.5193079556416222.
[I 2021-05-01 18:05:22,648] Trial 71 finished with value: 3.7071942474223105 and parameters: {'c': 1760258.879903194, 'gamma': 4.991942205918291e-06, 'epsilon': 0.5844335030672163}. Best is trial 44 with value: 3.5193079556416222.
[I 2021-05-01 18:06:51,549] Trial 72 finished with value: 3.586392242423006 a

[I 2021-05-01 18:33:01,239] Trial 102 finished with value: 3.617864204646417 and parameters: {'c': 1790788.7629338219, 'gamma': 4.892796329321162e-06, 'epsilon': 0.9380897959390456}. Best is trial 86 with value: 3.468563863301511.
[I 2021-05-01 18:34:51,337] Trial 103 finished with value: 3.513832456541412 and parameters: {'c': 2289720.9789106185, 'gamma': 2.6261562191926327e-06, 'epsilon': 0.6088880429444441}. Best is trial 86 with value: 3.468563863301511.
[I 2021-05-01 18:35:29,302] Trial 104 finished with value: 3.5166944748822924 and parameters: {'c': 998476.8565666643, 'gamma': 4.33866877247069e-06, 'epsilon': 0.8135697698988532}. Best is trial 86 with value: 3.468563863301511.
[I 2021-05-01 18:36:00,778] Trial 105 finished with value: 3.5452146968262443 and parameters: {'c': 750338.5583264701, 'gamma': 4.471686808327734e-06, 'epsilon': 0.49718897533087386}. Best is trial 86 with value: 3.468563863301511.
[I 2021-05-01 18:38:08,432] Trial 106 finished with value: 3.60571679156809

In [15]:
print('Melhores parâmetros:',cmaes.best_params)
print('RMSE:',cmaes.best_value)

Melhores parâmetros: {'c': 1762255.7593075274, 'gamma': 3.1031179228755386e-06, 'epsilon': 0.6423418528573434}
RMSE: 3.455354819319946
